In [2]:
import os
import warnings
warnings.filterwarnings("ignore")
from secret_key import openapi_key

os.environ['OPENAI_API_KEY'] = openapi_key

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [4]:
llm = ChatOpenAI()

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {input}")

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.predict(input="A pig")

'Why did the pig go to the casino? Because he heard they had a lot of "squeal" machines!'

### First pipe

In [7]:
chain = prompt | llm


In [8]:
chain.invoke({"input": "A pig"})

AIMessage(content='Why did the pig become an actor? Because he was great at hamming it up!')

In [9]:
from langchain.schema.output_parser import StrOutputParser

In [10]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "A pig"})

'Why did the pig go to the casino? Because he heard they had a lot of "squeal" machines!'

In [11]:
prompt = ChatPromptTemplate.from_template(
    "tell me 5 jokes about {input}"
    )

In [12]:
chain = prompt | llm.bind(stop=["\n"]) | StrOutputParser()

In [13]:
chain.invoke({"input": "pigs"})

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-GBzolOmzUHWBjRNhBNkzVVS1 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


'1. Why did the pig become an actor? Because he was always hamming it up!'

### OPENAI Functions

In [14]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]

In [15]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt 
    | llm.bind(function_call={"name": "joke"}, functions= functions) 
    | JsonOutputFunctionsParser()
)

In [17]:
chain.invoke(input={"input": "bears"})


{'setup': "Why don't bears like fast food?",
 'punchline': "Because they can't catch it!"}

### Working with vectorstores

In [18]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough


vectorstore = Chroma.from_texts(["Cats are typically 9.1 kg in weight.", 
                                 "Cats have retractable claws.", 
                                 "A group of cats is called a clowder.", 
                                 "Cats can rotate their ears 180 degrees.", 
                                 "The world's oldest cat lived to be 38 years old."], 
                                embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [19]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [20]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [21]:
chain.invoke("how old is the oldest cat?")

'The oldest cat lived to be 38 years old.'

In [22]:
from operator import itemgetter

In [23]:
question = {"bla": "test", "x": "wuff"}
itemgetter("bla")


operator.itemgetter('bla')

In [24]:
get_bla = itemgetter("bla")
get_bla(question)

'test'

In [25]:
from langchain.schema.runnable import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

# chain = {
#     "context": itemgetter("question") | retriever,
#     "question": itemgetter("question"),
#     "language": itemgetter("language")
# } | prompt | model | StrOutputParser()

chain = {
    "context": (lambda x: x["question"]) | retriever, 
    "question": (lambda x: x["question"]), 
    "language": (lambda x: x["language"])
} | prompt | llm | StrOutputParser()


In [26]:
chain.invoke({"question": "how old is the oldest cat?", "language": "german"})

'38 Jahre alt'

In [28]:
# !pip install duckduckgo-search

In [29]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [30]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [31]:
chain = prompt | llm | StrOutputParser() | search

In [33]:
print(chain.invoke({"input": "whats the name of the oldest cat?"}))

What is the oldest cat name? The earliest known cat with a name was called Nedjem, meaning 'sweet' or 'pleasant', and dates back to the reign of Thutmose III in Ancient Egypt (1479-1425 BC). Post navigation.


### Arbitrary functions

In [34]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | llm | StrOutputParser()

In [35]:
chain.invoke({"foo": "bar", "bar": "gah"})

'3 + 9 = 12'

### Interface

In [36]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [37]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="")

Why don't bears wear shoes?

Because they have bear feet!

In [39]:
chain.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they prefer to go bearfoot!")

In [42]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-GBzolOmzUHWBjRNhBNkzVVS1 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


[AIMessage(content="Why did the bear break up with his girlfriend?\n\nBecause he couldn't bear the relationship anymore!"),
 AIMessage(content='Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!')]

In [45]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="")
await chain.ainvoke({"topic": "bears"})
await chain.abatch([{"topic": "bears"}])

Why did the bear bring a flashlight to the party? Because he heard it was going to be a "bear"y bright night!

[AIMessage(content="Why don't bears wear shoes?\n\nBecause they prefer to go bear-foot!")]